### Install a Spark docker using the following commands

In [1]:
! docker pull bitnami/spark && \
docker network create spark_network && \
docker run -d --name spark --network=spark_network -e SPARK_MODE=master bitnami/spark
  

Using default tag: latest
latest: Pulling from bitnami/spark
Digest: sha256:45acd47b917751eb2f21b03cf4f21b567e44db7c78b2774f06c947caf5d3a9a6
Status: Image is up to date for bitnami/spark:latest
docker.io/bitnami/spark:latest
Error response from daemon: network with name spark_network already exists


### Create the Spark context to start a session and connect to the cluster.

In [2]:
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession, SQLContext
from pyspark.sql.types import *

def initspark(appname = "Notebook", servername = "local[*]"):
    print ('initializing pyspark')
    conf = SparkConf().setAppName(appname).setMaster(servername)
    sc = SparkContext(conf=conf)
    spark = SparkSession.builder.appName(appname).enableHiveSupport().getOrCreate()
    sc.setLogLevel("WARN")
    print ('pyspark initialized')
    return sc, spark, conf

sc, spark, conf = initspark()
print(sc, spark)

initializing pyspark


Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
21/12/23 02:39:13 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


pyspark initialized
<SparkContext master=local[*] appName=Notebook> <pyspark.sql.session.SparkSession object at 0x7faeefd1fed0>


### Push a local Python list into a Spark RDD and do a simple transformation.

In [5]:
rdd1 = ( sc.parallelize(['one', 'two', 'three', 'four'])
           .map(str.title)
       )
rdd1.collect()

['One', 'Two', 'Three', 'Four']

### Same code but with a lambda in case you don't have a built in function already.

In [6]:
rdd1 = ( sc.parallelize(['one', 'two', 'three', 'four'])
           .map(lambda x : x.title())
       )
rdd1.collect()

['One', 'Two', 'Three', 'Four']

### Read a text file from the local file system.

In [ ]:
shake = sc.textFile('/class/datasets/text/shakespeare.txt')
print(shake.count())
print(shake.take(10))


### Parallelize will load manually created data into the spark cluster into an RDD.

In [ ]:
r = sc.parallelize(range(1,11))
print(r.collect())
print(r.take(5))


### Load a folder stored on HDFS.

In [ ]:
cat = sc.textFile('hdfs://localhost:9000/categories')
print(cat.collect())


### Try some different actions to fetch data.

In [ ]:
print(cat.takeOrdered(5))
print(cat.top(5))
print(cat.takeSample(False,5))
cat.foreach(lambda x : print(x.upper)) # does not display properly in notebook


### Save the results in an RDD to disk. Note how it makes a folder and fills it with as many files as there are nodes solving the problem. Also, you must make sure that the folder does not exist or it throws an exception.

In [ ]:
! rm -r /class/file1.txt
cat.saveAsTextFile('hdfs://localhost:9000/file1.txt')


In [ ]:
! hadoop fs -ls /file1.txt


### Use the map method to apply a function call on each element.

In [ ]:
shake2 = shake.map(str.upper)
shake2.take(10)


### Using the split method you get a list of lists.

In [ ]:
shake3 = shake.map(lambda x : x.split(' '))
shake3.take(10)


In [ ]:
### The flatMap method flattens the inner list to return one big list of strings instead.

In [ ]:
shake4 = shake.flatMap(lambda x : x.split(' '))
shake4.take(20)


In [ ]:
print(cat.map(str.upper).collect())


### Parse the string into a tuple to resemble a record structure.

In [ ]:
cat1 = cat.map(lambda x : tuple(x.split(',')))
cat1 = cat1.map(lambda x : (int(x[0]), x[1], x[2]))
cat1.take(10)


## LAB: ## 
### Put the regions folder found in /class/datasets/northwind/CSV/regions into HDFS. Read it into an RDD and convert it into a tuple shape.
<br>
<details><summary>Click for <b>hint</b></summary>
<p>
Use hadoop fs -put or hdfs dfs -put
<br>
Read the file using sc.textFile
<br>
Do a map to split and another to convert the datatypes
<br>
<br>
</p>
</details>

<details><summary>Click for <b>code</b></summary>
<p>

```python
! hadoop fs -put /class/datasets/northwind/CSV/regions /regions
regions = sc.textFile('hdfs://localhost:9000/regions')
regions = regions.map(lambda x : x.split(',')).map(lambda x : (int(x[0]), x[1]))
print(regions.collect())
```
</p>
</details>

### You can chain multiple transformations together to do it all in one step.
#### Here we converted the datatypes to int, then turned the tuple into a dictionary.

In [ ]:
cat2 = cat.map(lambda x : tuple(x.split(','))) \
      .map(lambda x : (int(x[0]), x[1], x[2])) \
      .map(lambda x : dict(zip(['CategoryID', 'Name', 'Description'], x)))
cat2.take(10)


### The filter method takes a lambda that returns a True or False.

In [ ]:
cat2.filter(lambda x : x['CategoryID'] <= 5).collect()


### The filter expressions can be more complicated.

In [ ]:
cat2.filter(lambda x : x['CategoryID'] % 2 == 0 and 'e' in x['Name']).collect()


### The sortBy method returns an expression that is used to sort the data.

In [ ]:
cat2.sortBy(lambda x : x['Description']).collect()


### sortBy has an option ascending parameter to sort in reverse order.

In [ ]:
cat1.sortBy(lambda x : x[0], ascending = False).collect()


## LAB:##
### Try to sort region in descending order by ID and then by name in ascending order. ###

<br>
<details><summary>Click for <b>hint</b></summary>
<p>
Use sortByKey and sortBy respectively
<br>
sortBy needs a lambda
<br><br>
</p>
</details>

<details><summary>Click for <b>code</b></summary>
<p>

```python
print(regions.sortByKey(ascending = False).collect())
print(regions.sortBy(lambda x : x[1]).collect())
```
</p>
</details>

### The following are more complex examples of using Spark to do things like JOIN and GROUP BY. For the most part these methods are replaced by the newer DataFrame methods which we will explore in the next section. We will skip a detailed explanation of the following but leave it in for self study.

### Reshape categories from a tuple of three elements like (1, 'Beverages', 'Soft drinks') to a tuple with two elements (key, value) like (1, ('Beverages', 'Soft drinks')).

In [ ]:
cat3 = cat1.map(lambda x : (x[0], (x[1], x[2])))
cat3.collect()


### The sortByKey method does not require a function as a parameter if the data is structured into a tuple of the shape (key, value).

In [ ]:
cat3.sortByKey(ascending=False).collect()


### Read in another CSV file.

In [ ]:
prod = shake = sc.textFile('/class/datasets/northwind/CSV/products')
print(prod.count())
prod.take(4)


### Split it up and just keep the ProductID, ProductName, CategoryID, Price, Quantity values.

In [ ]:
prod1 = prod.map(lambda x : x.split(',')).map(lambda x : (int(x[0]), x[1], int(x[3]), float(x[5]), int(x[6])))
prod1.take(5)


### Reshape it to a key value tuple where category is the key and the other fields are the values.

In [ ]:
prod2 = prod1.map(lambda x : (x[2], (x[0], x[1], x[3], x[4])))
prod2.take(5)


In [ ]:
cat3.collect()


### Both c3 and prod2 are in key value tuple format so they can be joined to produce a new tuple of (key, (cat, prod)).

In [ ]:
joined = cat3.join(prod2)
joined.sortByKey().take(15)


## LAB: ##
### Load territories into HDFS and join it to regions. ###


<br>
<details><summary>Click for <b>hint</b></summary>
<p>
Put /class/datasets/northwind/CSV/territories into HDFS
<br>
Use sc.textFile to read it into an RDD
<br>
Use map to split and convert it to the proper datatypes
<br>
Use the join method
<br><br>
</p>
</details>

<details><summary>Click for <b>code</b></summary>
<p>

```python
! hadoop fs -put /class/datasets/northwind/CSV/territories /

territories = sc.textFile('hdfs://localhost:9000/territories')
territories = territories.map(lambda x : x.split(',')).map(lambda x : (int(x[0]), x[1], int(x[2])))
print(territories.collect())

region_territories = regions.join(territories.map(lambda x : (x[2], (x[0],x[1]))))
print(region_territories.collect())
# Reshape it to make it look more normal. The * in front of the x is a python unpacking trick
region_territories = region_territories.map(lambda x : (x[0], (x[1][0], *x[1][1])))
print(region_territories.collect())
```
</p>
</details>

### The groupBy methods are seldom used but they can produce hierarchies where children records are embedded inside a parent.

In [ ]:
group1 = prod2.groupByKey()
group1.take(3)


In [ ]:
list(group1.take(1)[0][1])


In [ ]:
group2 = [(key, list(it)) for key, it in group1.collect()]
for k,v in group2:
    print ('Key:', k)
    for x in v:
        print(x)
#print (group2)


### The reduce methods take a function as a parameter that tells Spark how to accumulate the values for each group. The function takes two parameters; the first is the accumulated value and the second is the next value in the list. 

In [ ]:
shake4.map(lambda x : (x, 1)).reduceByKey(lambda x, y : x + y).sortBy(lambda x : x[1], ascending = False).take(10)


## LAB: ## 
### Use the territories RDD to count how many territories are in each region. 
### Display the results in regionID order and then descending order based on the counts.
<br>
<details><summary>Click for <b>hint</b></summary>
<p>
Use map to put the key first then reduceByKey to accumulate the values
<br>
Use sortByKey to sort by regionID and sortBy with a lambda to sort by counts
<br><br>
</p>
</details>

<details><summary>Click for <b>code</b></summary>
<p>

```python
region_count = territories.map(lambda x : (x[2], 1)).reduceByKey(lambda x, y: x + y)
print(region_count.sortByKey().collect())
print(region_count.sortBy(lambda x : x[1], ascending = False).collect())
```
</p>
</details>

### In this example, we are adding up all the prices for each categoryID.

In [ ]:
red1 = prod2.map(lambda x : (x[0], x[1][2])).reduceByKey(lambda x, y: x + y)
red1.collect()


### To accumulate more than one value, use a tuple to hold as many values as you want to aggregate.

In [ ]:
red1 = prod2.map(lambda x : (x[0], (x[1][2], x[1][3], 1))).reduceByKey(lambda x, y: (x[0] + y[0], x[1] + y[1], x[2] + y[2]))
red1.collect()


### Some Python magic can make things easier in the long run.
Named tuples make accessing the elements of the row easier.
Unpacking using the * is a neat Python trick that is widely used.
 
datetime has function to convert a string into a date.

In [ ]:
mort = sc.textFile('/class/datasets/finance/30YearMortgage.csv')
head = mort.first()
mort = mort.filter(lambda x : x != head)


In [ ]:
from datetime import date, datetime
from collections import namedtuple
Rate = namedtuple('Rate','date fed_fund_rate avg_rate_30year')
mort1 = mort.map(lambda x : Rate(*(x.split(','))))
mort2 = mort1.map(lambda x : Rate(datetime.strptime(x.date, '%Y-%m').date(), float(x.fed_fund_rate), float(x.avg_rate_30year)))
mort2.take(5)


In [ ]:
mort2.filter(lambda x : x.fed_fund_rate > .1 ).collect()


### HOMEWORK:
1. The creditcard.csv dataset provides sample data on credit card transactions.
2. Load the file into HDFS.
3. Load the file into an RDD.
4. Parse the file into a tuple or namedtuple or dictionary.
5. Make sure to convert columns to the right data types.
6. You can ignore any columns you don’t need for the solution.
7. Filter the data to show only transactions made by women.
8. Calculate the amount spent in each city.
